In [1]:
#Daily Payments Processed for Vendors - in CARS Financial Folder
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment, PatternFill
import os

# Paths
base_path = Path(r"Z:\work\ach_eobr\DailyArchive")
mapping_path = Path(r"Z:\work\ach_eobr\DataSource\VendorList.csv")
log_file_path = base_path / 'error_log.txt'

# Load mapping data
mapping_data = pd.read_csv(mapping_path)
filtered_data = pd.read_excel('Daily Payments Processed for Vendors (194).xlsx', skiprows=3, header=0)
filtered_data = filtered_data.drop('Unnamed: 0', axis=1)
filtered_data["Check #"] = filtered_data["Check #"].astype(str)
filtered_data = filtered_data[~filtered_data["Check #"].str.startswith("0")]
data_copy = filtered_data

# Function to get the previous business day
def get_previous_business_day():
    result = datetime.now()
    if result.weekday() == 0:  # Monday
        result = result - timedelta(days=3)
    else:
        result = result - timedelta(days=1)
    return result

# Prepare date information
previous_business_day = get_previous_business_day()
formatted_previous_day = previous_business_day.strftime('%Y-%m-%d')

start_date = formatted_previous_day
end_date = formatted_previous_day

datestamp = previous_business_day.strftime('%Y%m%d')
date_folder = base_path / datestamp

#Filter out data that is not from previous business day.
filtered_data = filtered_data[(filtered_data['Transaction Date'] >= start_date) & (filtered_data['Transaction Date'] <= end_date)]


#FORMATTING
filtered_data['Service Start'] = pd.to_datetime(filtered_data['Service Start'])
filtered_data['Service Start'] = filtered_data['Service Start'].dt.strftime('%Y-%m-%d')
filtered_data['Service End'] = pd.to_datetime(filtered_data['Service End'])
filtered_data['Service End'] = filtered_data['Service End'].dt.strftime('%Y-%m-%d')
filtered_data['Service End'] = pd.to_datetime(filtered_data['Service End'])
filtered_data['Service End'] = filtered_data['Service End'].dt.strftime('%Y-%m-%d')
filtered_data['Transaction Date'] = pd.to_datetime(filtered_data['Transaction Date'])
filtered_data['Transaction Date'] = filtered_data['Transaction Date'].dt.strftime('%Y-%m-%d')
filtered_data['Payee Tax ID'] = filtered_data['Payee Tax ID'].astype(str)
mapping_data['VendorTaxID'] = mapping_data['VendorTaxID'].astype(str)
filtered_data['Payee Tax ID'] = filtered_data['Payee Tax ID'].str.split('.').str[0]
mapping_data['VendorTaxID'] = mapping_data['VendorTaxID'].str.split('.').str[0]

# Enrich filtered_data with VendorCode from mapping_data
filtered_data = filtered_data.merge(
    mapping_data[['VendorTaxID', 'VendorCode']],
    how='left',
    left_on='Payee Tax ID',
    right_on='VendorTaxID'
)

#Dropping Unneeded
filtered_data['VendorCode'].fillna('NONSPECIALTY_VENDOR_PAYMENTS', inplace=True)
filtered_data.drop(columns=['Payee Tax ID'], inplace=True)
filtered_data.drop(columns=['Check #'], inplace=True)

# Ensure date_folder exists
if not date_folder.exists():
    raise Exception(f"Date folder {date_folder} does not exist.")

# Function to sanitize file names
def sanitize_filename(name):
    return name.replace('/', '_').replace('\\', '_').replace(':', '_').replace('*', '_')\
        .replace('?', '_').replace('"', '_').replace('<', '_').replace('>', '_').replace('|', '_')

# Open log file
with open(log_file_path, 'w') as log_file:
    for payee, group in filtered_data.groupby('Payee'):
        try:
            vendor_code = group['VendorCode'].iloc[0]
            
            # Determine target folder
            if vendor_code == "NONSPECIALTY_VENDOR_PAYMENTS":
                target_folder = date_folder / "NONSPECIALTY_VENDOR_PAYMENTS"
                # Create OTHER_VENDOR_PAYMENTS folder only if needed
                if not target_folder.exists():
                    target_folder.mkdir(parents=True, exist_ok=True)
            else:
                target_folder = date_folder / vendor_code

            # Ensure the target folder exists
            if not target_folder.exists():
                raise Exception(f"Folder for vendor '{vendor_code}' does not exist.")

            sanitized_payee = sanitize_filename(payee)
            file_path = target_folder / f"{sanitized_payee}_transactions_{datestamp}.xlsx"

            # Export to Excel
            group.to_excel(file_path, index=False)

            # Format Excel file
            wb = load_workbook(file_path)
            ws = wb.active

            # Style headers
            header_font = Font(bold=True, color="FFFFFF")
            header_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
            header_alignment = Alignment(horizontal="center", vertical="center")

            for cell in ws[1]:
                cell.font = header_font
                cell.fill = header_fill
                cell.alignment = header_alignment

            # Adjust column widths
            for col in ws.columns:
                max_length = 0
                col_letter = get_column_letter(col[0].column)
                for cell in col:
                    if cell.value is not None:
                        max_length = max(max_length, len(str(cell.value)))
                ws.column_dimensions[col_letter].width = max_length + 2

            # Add total row
            last_row = ws.max_row + 1
            ws.cell(row=last_row, column=8).value = 'Total'
            ws.cell(row=last_row, column=9).value = group['Amount'].sum()

            # Save workbook
            wb.save(file_path)

        except Exception as e:
            # Log errors
            log_file.write(f"Error processing payee '{payee}': {str(e)}\n")
            print(f"Error processing payee '{payee}': {str(e)}")

print("Process completed. Check the error log for details.")


C:\Users\gduln001\AppData\Local\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\gduln001\AppData\Local\Temp\2\ipykernel_33024\30753602.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Service Start'] = pd.to_datetime(filtered_data['Service Start'])
C:\Users\gduln001\AppData\Local\Temp\2\ipykernel_33024\30753602.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

Process completed. Check the error log for details.


In [2]:
filtered_data

,Payee,Transaction Date,Claimant Name,Claim Number,Invoice Number,Bill Review Bill ID,Service Start,Service End,Amount,VendorTaxID,VendorCode
0,LARJAR INC,2025-04-16,"Lingo, Brenda G",WDWW2024239398,NaN,99997-X-13293141-1,2025-01-03,2025-01-27,362.03,208185083,PROCARE
1,SELECT PROVIDER NETWORKS INC,2025-04-16,"Ensign, Michael R",DLRW2024102899,PAC117071,99997-H-13364298-0,2025-03-19,2025-03-19,106.00,232935684,DBA SPNET
2,SELECT PROVIDER NETWORKS INC,2025-04-16,"Godinez, Diana A",DLRW2024101922,140627553,99997-H-13364306-0,2025-01-14,2025-01-14,106.00,232935684,DBA SPNET
3,SELECT PROVIDER NETWORKS INC,2025-04-16,"Gonzalez, Irma N",DLRW2024103966,654367Z4873,99997-H-13367953-0,2025-03-25,2025-03-25,106.00,232935684,DBA SPNET
4,SELECT PROVIDER NETWORKS INC,2025-04-16,"Han, Eleanor H",DLRW2017080458,16906599A,99997-H-13367959-0,2025-03-26,2025-03-26,106.00,232935684,DBA SPNET
...,...,...,...,...,...,...,...,...,...,...,...
272,KIMBERLY J EVANS OTR/L CHT PC,2025-04-16,"Walujono, Anthony J",DLRW2025104307,102404,99997-H-13361231-0,2025-04-03,2025-04-03,161.32,821348823,KIMBERLY_EVANS
273,JEWETT ORTHOPEDIC INSTITUTE SURGERY,2025-04-16,"Lavache, Carole",WDWW2024238136,NaN,99997-U-13338128-0,2025-03-12,2025-03-12,9478.96,872667237,JWT_SURG
274,DOCTORS HOSPITAL BAHAMAS LTD,2025-04-16,"Ramsli, Hanne Marte",DCLW2025312128,VAA018314260,99997-U-13357272-0,2025-02-20,2025-02-20,1387.91,980160433,DOCTORS_HOSPITAL
275,CENTRO MEDICO DE COZUMEL S.A. DE C.,2025-04-16,"De Santiago Arredondo, Rodrigo",DCLW2025004332,226831,99997-Y-13348804-0,2025-02-10,2025-02-10,829.99,NaN,NONSPECIALTY_VENDOR_PAYMENTS
